In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from peak_detection import detect_peaks_rust_locally_exclusive_on_chunk

In [11]:
import spikeinterface.full as si
# import spikeinterface.sortingcomponents.peak_detection as pkd
import numpy as np
import time



# import sys
# sys.path.append("/home/samuel/Documents/SpikeInterface/spikeinterface_rust/Python/spikeinterface_files")
from main_detect import detect_peaks

In [20]:
job_kwargs={"n_jobs": 4, "mp_context": "fork", "pool_engine":"process", "progress_bar": True, "chunk_duration":"1s"}

In [21]:
rec, sorting = si.generate_ground_truth_recording(durations=[100],num_channels=384,sampling_frequency=25000.0,num_units=500, seed=2205)
rec = rec.save_to_memory(**job_kwargs)

write_memory_recording (workers: 4 processes):   0%|          | 0/100 [00:00<?, ?it/s]

In [27]:

job_kwargs={"n_jobs": 4, "mp_context": "fork", "pool_engine":"process", "progress_bar": True, "chunk_duration":"1s"}
# job_kwargs={"n_jobs": 1, "mp_context": "fork", "pool_engine":"process", "progress_bar": True, "chunk_duration":"1s"}
# job_kwargs={"n_jobs": 4, "mp_context": "fork", "pool_engine":"thread", "progress_bar": True, "chunk_duration":"1s"}

peaks_numba = detect_peaks(rec, method="locally_exclusive", method_kwargs={"engine": "numba"}, job_kwargs=job_kwargs)

peaks_rust = detect_peaks(rec, method="locally_exclusive", method_kwargs={"engine": "rust"}, job_kwargs=job_kwargs)

detect peaks (locally_exclusive) (workers: 4 processes):   0%|          | 0/100 [00:00<?, ?it/s]

detect peaks (locally_exclusive) (workers: 4 processes):   0%|          | 0/100 [00:00<?, ?it/s]

In [31]:
print(peaks_numba.size, peaks_rust.size, np.array_equal(peaks_numba, peaks_rust))
print(peaks_numba[-10:-1])
print(peaks_rust[-10:-1])

625161 625161 True
[(2499963,  61,  -41.60811615, 0) (2499971, 189,  -34.10840607, 0)
 (2499971, 374,  -42.52897263, 0) (2499980, 284, -206.00794983, 0)
 (2499981,   5,  -54.36881256, 0) (2499984, 146, -104.21274567, 0)
 (2499992,  51, -258.73120117, 0) (2499994, 101,  -75.33312225, 0)
 (2499995, 164,  -34.02312088, 0)]
[(2499963,  61,  -41.60811615, 0) (2499971, 189,  -34.10840607, 0)
 (2499971, 374,  -42.52897263, 0) (2499980, 284, -206.00794983, 0)
 (2499981,   5,  -54.36881256, 0) (2499984, 146, -104.21274567, 0)
 (2499992,  51, -258.73120117, 0) (2499994, 101,  -75.33312225, 0)
 (2499995, 164,  -34.02312088, 0)]


In [32]:
np.setdiff1d(peaks_numba, peaks_rust)

array([],
      dtype=[('sample_index', '<i8'), ('channel_index', '<i8'), ('amplitude', '<f8'), ('segment_index', '<i8')])

In [ ]:
# for i in range(1,4):
#     job_kwargs={"n_jobs": -1, "mp_context": "fork", "pool_engine":"thread", "progress_bar": False, "chunk_duration": str(500*i) +"ms"}
#     print("Chunk duration :" + str(500*i) +" ms")
#     %timeit pkd.detect_peaks(rec, method="locally_exclusive", method_kwargs={"engine": "rust"}, job_kwargs=job_kwargs)

Chunk duration :100ms
249 ms ± 6.64 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Chunk duration :200ms
132 ms ± 2.16 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Chunk duration :300ms
95.6 ms ± 1.15 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Chunk duration :400ms
80.2 ms ± 862 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Chunk duration :500ms
70.7 ms ± 870 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Chunk duration :600ms
64.9 ms ± 591 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Chunk duration :700ms
61.4 ms ± 850 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Chunk duration :800ms
59.2 ms ± 1.08 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Chunk duration :900ms
56.9 ms ± 969 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Chunk duration :1000ms
56.8 ms ± 1.89 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Chunk duration :1100ms
55.5 ms ± 760 μs per loop (mean ± std. dev. of 7 runs, 

In [ ]:
for i in range(1,4):
    job_kwargs={"n_jobs": -1, "mp_context": "fork", "pool_engine":"thread", "progress_bar": False, "chunk_duration": str(500*i) +"ms"}
    print("Chunk duration :" + str(500*i) +" ms")
    %timeit pkd.detect_peaks(rec, method="locally_exclusive", method_kwargs={"engine": "numba"}, job_kwargs=job_kwargs)

Chunk duration :500ms
81.1 ms ± 915 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Chunk duration :1000ms
85.5 ms ± 1.14 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Chunk duration :1500ms
107 ms ± 1.48 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
